# Домашна работа 
## Теорија
Објаснување на формулите

За да филтрираме слика во фреквенциски домен потребно е да ја помножиме сликата во фреквенциски домен со филтерот ( кернел ) кој е исто така во фреквенциски домен.
<br><br>

\begin{equation}
I_{f,filt} = HI_f
\end{equation}


За таа цел потребно е да се вметне Octave функција која ќе креира “Band pass gaussian filter” во фреквенциски домен. Bandpass гаусовиот филтер во фреквенциски домен е дефиниран со равенката


\begin{equation}
H(i,j) = e ^{-\frac{d(i,j)^2}{f^2_l}} \left( 1 - e ^{-\frac{d(i,j)^2}{f^2_l}} \right)\
\end{equation}

Каде што $d(i,j)$ е евклидовото растојанието на даден пиксел од центарот на сликата, додека $f_h$ и $f_l$ се праговите на високите и ниските фреквенции кои сакаме да ги филтрираме. 
<br> <br>
Конверзија на 2D сликата од просторен во фреквенциски домен се прави со користење на **Фуриева трансформација**.Фуриевата трансформација е дефинирана како: 

\begin{equation*}
F(u,v) = \iint_{-\infty}^{\infty} f(x,y) e^{-i2\pi(ux+vy)} dxdy
\end{equation*}

А во дискретниот домен горната равенка може да биде преведена како 

\begin{equation}
F(u,v) = \displaystyle\sum_{m=-\infty}^{\infty} \sum_{n=-\infty}^{\infty} f[m,n] \cdot e^ {-i2\pi(umx_0+vny_0)}
\end{equation}


Конверзија на 2D сликата од фреквенциски во просторен домен се прави со користење на инверзна Фуриева трансформација. Инверзната Фуриева трансформација е дефинирана како:

\begin{equation*}
F(x,y) = \iint_{-\infty}^{\infty} f(u,v) e^{i2\pi(ux+vy)} dudv
\end{equation*}

А во дискретниот домен горната равенка може да биде преведена како

\begin{equation}
F(x,y) = \displaystyle\sum_{m=-\infty}^{\infty} \sum_{n=-\infty}^{\infty} f[m,n] \cdot e^ {i2\pi(xmu_0+ynv_0)}
\end{equation}

Во следната келија Octave се извршуваат следните работи:
    - Креирање на филтер со помош на create_filter (nx,ny,d0,d1), каде nx и ny се димензиите на сликата a d0 и d1 се фреквенциски прагови кои се филтерираат,
    - Читање на слика и нејзини димензии 
    - Трансформирање на сликата од просторен во фреквенциски домен
    - Креирање на Гаусов филтер во фреквенциски домен и филтрирање на слика
    - Креирање на bandpass кернел 
    - Филтрирање на слика со bandpass кернел и трансформирање назад во домен
    - Зачувуање на матриците filtered_kernel и filtered_image

In [1]:
% pkg load image

% function filter = create_filter(nx,ny,d0,d1)
% filter = ones(nx,ny);
% for i = 0:nx-1
%     for j = 0:ny-1
%         dist= sqrt((i-nx/2)^2 + (j-ny/2)^2);
%         filter(i+1,j+1) = exp(-(dist^2)/(d1^2)).*(1.0-exp(-(dist^2)/(d0^2)));
%      end
% end
% filter(nx/2+1,ny/2+1)=1;
% end

% %read image
% im = double(imread('Barbara.tif'));
% %size of the image
% [nx ny] = size(im);
% %Transform the image from spatial to frequency domain
% im_fft = fftshift(fft2(im));
% % Create Gaussian bandpass kernels, in frequency domain, for 20 different cut off frequencies and filter the image
% p=1;
% for i = 1:10:101
% q=1;
%     for j = 1:10:101
%         %Create the bandpass kernel
%         tmp1 = create_filter(nx,ny,i,j);
%         %Filter the image with the bandpass kernel and transform it back to spatial domain
%         tmp = mat2gray(abs(ifft2(ifftshift(tmp1.*im_fft))));
%         %Save the filtered image (in spatial domain) and kernel (in frequency domain) for visualization 
%         filtered_image(p,q,:) = tmp(:);
%         filter_kernel(p,q,:) = tmp1(:);
%         q=q+1;
%     end
%     p=p+1;
% end
% save filter_matrix.mat filter_kernel 
% save filter_image.mat filtered_image

Читање на матриците со помош на Oct2py 

In [2]:
from oct2py import octave

matrix_1 = octave.load('filter_matrix.mat')
matrix_2 = octave.load('filter_image.mat')

Земање на соодведни вредности од читањето на .mat филовите

In [3]:
filters = matrix_1['filter_kernel']
images = matrix_2['filtered_image']

Во images матрицата, сликата која што одговара на нефилтрираната слика ( Barbara.tif) е во ред 0 и колона 10 истата ја читаме и ја ротираме.

In [6]:
import numpy as np
tmp = images[0][10]
tmp = np.flip(octave.reshape(tmp,566,402))

Креирање на фигура каде со менување на вредности на слајдерот се менуваат праговите на филтрирањето

In [7]:
from ipywidgets import widgets, VBox, interactive
import plotly.graph_objs as go
import numpy as np

slider = widgets.IntRangeSlider(
    value=[1, 101],
    min=1,
    max=101,
    step=10,
    description='Freq',
    continuous_update=False,
    readout=True,
    readout_format='d'
)
slider.layout.width = '750px'

fig = go.FigureWidget()

fig.add_heatmap(z=tmp,
                colorscale = 'Gray',
                showscale = False)

fig.layout.title='Filter image'
fig.layout.autosize = True

fig.update_xaxes(visible = False)
fig.update_yaxes(visible = False)
fig.update_layout(height = 700, width = 550)
fig.update_traces(overwrite = True)

def up_func(current):
    index_x = int(np.floor(slider.value[0]/10))
    index_y = int(np.floor(slider.value[1]/10))
    tmp = images[index_x][index_y]
    tmp = np.flip(octave.reshape(tmp,566,402))
    fig.data[0]['z'] = tmp 

vb = VBox((interactive(up_func, current = slider), fig))
vb.layout.align_items = 'center'
vb